In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import requests
import sys
import os

In [2]:
raw_df = pd.read_html("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Director")[2]

In [3]:
director_df = raw_df[raw_df["Year"]!=raw_df["Film"]]

In [4]:
list_of_years = director_df.Year.unique()

In [5]:
years = []
director=[]
film=[]


for x in list_of_years:
    years.append(director_df[director_df["Year"]==x].iloc[0]["Year"])
    director.append(director_df[director_df["Year"]==x].iloc[0]["Director(s)"])
    film.append(director_df[director_df["Year"]==x].iloc[0]["Film"])

In [6]:
directors = []

for i in director:
    directors.append(i.split("&"))

In [7]:
directors_clean =[]

for i in directors:
    directors_clean.append(i[0])

In [8]:
for i in range(len(directors_clean)):
    directors_clean[i] = re.sub("[\(\[].*?[\)\]]", "", directors_clean[i])

In [9]:
url=[]
for i in directors_clean:
    url.append(f"https://en.wikipedia.org/wiki/{i}")

In [10]:
bestdirector_df = pd.DataFrame({"Years": years, "Winner Name":directors_clean, "Film": film, "Category":"Best Director", "Wiki URL":url})
bestdirector_df[["Year","Oscar Year"]] = bestdirector_df.Years.str.split("(",expand=True)
bestdirector_df["Oscar Year"] = bestdirector_df["Oscar Year"].str.replace(r"[\(\[].*?[\)\]]","")
bestdirector_df["Oscar Year"] = bestdirector_df["Oscar Year"].str.replace(")","")
# bestdirector_df["Year"] = bestdirector_df["Year"].str.replace(r"\(.*?\)","")
bestdirector_df = bestdirector_df[['Year','Oscar Year','Winner Name','Film', "Category",'Wiki URL']]

In [88]:
birthplace =[]
birthyear=[]
imageURL=[]


for x in url:
    response = requests.get(x)
    soup = bs(response.text, 'html.parser')
    try:
        birthplace_result = soup.find_all('div', class_='birthplace')[0].text
        birthplace.append(birthplace_result)
    except: 
        print("Oops!",sys.exc_info()[0],"occured")
        birthplace.append('Unknown')
    try:
        birthyear_result = soup.find_all('span', class_='bday')[0].text[:4]
        birthyear.append(birthyear_result)
    except:
        print("Oops!",sys.exc_info()[0],"occured")
        birthyear.append('Unknown')

Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured
Oops! <class 'IndexError'> occured


In [22]:
response = requests.get('https://en.wikipedia.org/wiki/Frank_Lloyd')
soup = bs(response.text, 'html.parser')
imageURL_result = soup.find_all('table', class_='biography')[0].find_all('a')[0].get('href')
imageURL_result

'/wiki/File:Frank_Lloyd,_Boxoffice_Barometer,_1939.jpg'

In [90]:
bestdirector_df['Birthplace']=birthplace
bestdirector_df["Birth Year"]=birthyear
bestdirector_df.to_csv("bestdirector.csv",index=False)
bestdirector_df.head(20)

,Year,Oscar Year,Winner Name,Film,Category,Wiki URL,Birthplace,Birth Year
0,1927/28,1st,Frank Borzage,7th Heaven,Best Director,https://en.wikipedia.org/wiki/Frank Borzage,"Salt Lake City, Utah, U.S.",1894
1,1928/1929,2nd,Frank Lloyd,The Divine Lady,Best Director,https://en.wikipedia.org/wiki/Frank Lloyd,"Glasgow, Scotland, UK",Unknown
2,1929/1930,3rd,Lewis Milestone,All Quiet on the Western Front,Best Director,https://en.wikipedia.org/wiki/Lewis Milestone,"Chişinău, Bessarabia, Russian Empire (now Mold...",1895
3,1930/1931,4th,Norman Taurog,Skippy,Best Director,https://en.wikipedia.org/wiki/Norman Taurog,"Chicago, Illinois, United States",1899
4,1931/1932,5th,Frank Borzage,Bad Girl,Best Director,https://en.wikipedia.org/wiki/Frank Borzage,"Salt Lake City, Utah, U.S.",1894
5,1932/1933,6th,Frank Lloyd,Cavalcade,Best Director,https://en.wikipedia.org/wiki/Frank Lloyd,"Glasgow, Scotland, UK",Unknown
6,1934,7th,Frank Capra,It Happened One Night,Best Director,https://en.wikipedia.org/wiki/Frank Capra,"Bisacquino, Italy",1897
7,1935,8th,John Ford,The Informer,Best Director,https://en.wikipedia.org/wiki/John Ford,"Cape Elizabeth, Maine, U.S.",1894
8,1936,9th,Frank Capra,Mr. Deeds Goes to Town,Best Director,https://en.wikipedia.org/wiki/Frank Capra,"Bisacquino, Italy",1897
9,1937,10th,Leo McCarey,The Awful Truth,Best Director,https://en.wikipedia.org/wiki/Leo McCarey,"Los Angeles, California, U.S.",1898
